In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
IMAGE_SIZE = 224
BATCH_SIZE = 3

datagen = ImageDataGenerator(
    rescale=1./255.,
    validation_split = 0.2
)

train_data = datagen.flow_from_directory('./drive-download-20230601T031056Z-001/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, subset='training')
validation_data = datagen.flow_from_directory('./drive-download-20230601T031056Z-001/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE, subset='validation')

Found 166 images belonging to 10 classes.
Found 41 images belonging to 10 classes.


In [22]:
feature_extractor_url = 'https://tfhub.dev/google/efficientnet/b0/feature-vector/1'
feature_extractor_layer = hub.KerasLayer(feature_extractor_url, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
feature_extractor_layer.trainable = False

In [23]:
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.LayerNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [24]:
steps_per_epoch = np.ceil(train_data.samples / train_data.batch_size)
history = model.fit(train_data, epochs=10, steps_per_epoch=steps_per_epoch)

Epoch 1/10
56/56 [==============================] - 18s 191ms/step - loss: 2.4686 - accuracy: 0.2590
Epoch 2/10
56/56 [==============================] - 10s 186ms/step - loss: 1.0961 - accuracy: 0.6446
Epoch 3/10
56/56 [==============================] - 11s 188ms/step - loss: 0.6056 - accuracy: 0.8133
Epoch 4/10
56/56 [==============================] - 11s 190ms/step - loss: 0.4281 - accuracy: 0.8735
Epoch 5/10
56/56 [==============================] - 11s 188ms/step - loss: 0.2907 - accuracy: 0.9639
Epoch 6/10
56/56 [==============================] - 11s 188ms/step - loss: 0.2246 - accuracy: 0.9699
Epoch 7/10
56/56 [==============================] - 11s 192ms/step - loss: 0.1641 - accuracy: 1.0000
Epoch 8/10
56/56 [==============================] - 10s 185ms/step - loss: 0.1253 - accuracy: 1.0000
Epoch 9/10
56/56 [==============================] - 10s 183ms/step - loss: 0.1033 - accuracy: 1.0000
Epoch 10/10
56/56 [==============================] - 11s 189ms/step - loss: 0.0840 - accura

In [26]:
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

array(['Acne', 'Actinic Keratosis', 'Basa Cell Carcinoma', 'Eksim',
       'Flek Hitam', 'Garis Wajah Dan Kerutan', 'Herpes', 'Milia',
       'Rosacea', 'Vitiligo'], dtype='<U23')

In [27]:
for image_batch, label_batch in validation_data:
    predicted_batch = model.predict(image_batch)
    predicted_id = np.argmax(predicted_batch, axis=-1)
    predicted_label_batch = class_names[predicted_id]
model.evaluate(validation_data)

1/1 [==============================] - 0s 468ms/step


KeyboardInterrupt: 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('skin.tflite', 'w') as f:
    f.write(tflite_model)

In [28]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [39]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def main():
    image_path = './drive-download-20230601T031056Z-001/1XML/'
    xml_df = xml_to_csv(image_path) 
    xml_df.to_csv('label.csv', index=None) #nama output csv
    print('Successfully converted xml to csv.')

In [40]:
main()

Successfully converted xml to csv.


In [1]:
!pip install tflite_model_maker

  Using cached tflite_model_maker-0.4.2-py3-none-any.whl (577 kB)
  Using cached tf_models_official-2.3.0-py2.py3-none-any.whl (840 kB)
  Using cached sentencepiece-0.1.99-cp38-cp38-win_amd64.whl (977 kB)
  Using cached tensorflow_datasets-4.9.2-py3-none-any.whl (5.4 MB)
  Using cached fire-0.5.0.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached tensorflowjs-3.18.0-py3-none-any.whl (77 kB)
  Using cached tensorflow-2.12.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached numba-0.53.0-cp38-cp38-win_amd64.whl (2.3 MB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached matplotlib-3.4.3-cp38-cp38-win_amd64.whl (7.1 MB)
  Using cached tensorflow_addons-0.20.0-cp38-cp38-win_amd64.whl (746 kB)
  Using cached neural_structured_learning-1.4.0-py2.py3-none-any.whl (128 kB)
  Using cached tensorflow_model_optimization-0.7.5-py2.py3-none-an

ERROR: Cannot install flax because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [5]:
from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

ModuleNotFoundError: No module named 'tflite_model_maker'

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'android_figurine/train',
    'android_figurine/train',
    ['android', 'pig_android']
)